# Subgraph Isomorphism

### Useful References

* [Subgraph Isomorphism Problem](https://en.wikipedia.org/wiki/Subgraph_isomorphism_problem#CITEREFUllmann2010)
* [Original Algorithm](https://dl.acm.org/doi/pdf/10.1145/321921.321925)

In [1]:
from graph import Graph

from queue import LifoQueue
from copy import deepcopy

import math as m

### All possible subgraphs

![K3inK6](./media/videos/subgraph_search/720p30/K3inK6.gif "K3 in K6")

More GIFs:
* [K4 in K6](./media/videos/subgraph_search/720p30/K4inK6.gif)
* [K4 in K8](./media/videos/subgraph_search/720p30/K4inK8.gif)
* [K9 in K10](./media/videos/subgraph_search/720p30/K9inK10.gif)

### Choosing Objects

In [2]:
def enumerate_arrangements(n: int) -> list:
    arrangements = []
    stack = LifoQueue()
    stack.put([])

    while not stack.empty():
        item = stack.get()

        if len(item) == n:
            arrangements.append(item)
        else:
            for i in range(n):
                if not i in item:
                    new_item = deepcopy(item)
                    new_item.append(i)
                    stack.put(new_item)

    return arrangements

In [3]:
def enumerate_choices(n: int, r: int) -> list:
    choices = []
    stack = LifoQueue()
    stack.put([])

    while not stack.empty():
        item = stack.get()

        if len(item) == r:
            choices.append(item)
        else:
            for i in range((item[-1] + 1) if len(item) > 0 else 0, n):
                new_item = deepcopy(item)
                new_item.append(i)
                stack.put(new_item)

    return choices

In [18]:
n = 2
r = 2

arrangements = enumerate_arrangements(n)
choices = enumerate_choices(n, r)

print(len(arrangements), arrangements)
print(len(choices), choices)

assert(len(arrangements) == m.factorial(n))
assert(len(choices) == m.comb(n, r))

2 [[1, 0], [0, 1]]
1 [[0, 1]]


### Searching for a subgraph

In [78]:
def subgraph_search(G: Graph, H: Graph):
    stack = LifoQueue()
    stack.put([])

    while not stack.empty():
        item = stack.get()

        if len(item) == H.properties.order:
            local_result = True
            for vert_a_H, vert_a_G in enumerate(item):
                for vert_b_H, vert_b_G in enumerate(item[vert_a_H + 1:]):
                    vert_b_H += vert_a_H + 1
                    if vert_a_G == vert_b_G or H[vert_a_H][vert_b_H] == -1:
                        continue

                    local_result = local_result and (G[vert_a_G][vert_b_G] == H[vert_a_H][vert_b_H])

            if local_result:
                return True
        else:
            for i in range((item[-1] + 1) if len(item) > 0 else 0, G.properties.order):
                new_item = deepcopy(item)
                new_item.append(i)
                stack.put(new_item)

    return False

### Subgraph Examples

All $P_n \subseteq C_n \subseteq K_n$ 

In [85]:
path_2 = Graph.subgraph_path_n(2)
cycle_2 = Graph.subgraph_cycle_n(2)
complete_2 = Graph.complete_n(2)
assert(subgraph_search(cycle_2, complete_2)) # Check that K_2 in C_2
assert(subgraph_search(path_2, cycle_2)) # Check that C_2 in P_2

cycle_3 = Graph.subgraph_cycle_n(3)
complete_3 = Graph.complete_n(3)
assert(subgraph_search(cycle_3, complete_3)) # Check that K_3 in C_3

# Start at 4 because C_3 = K_3 and P_2 = C_2 = K_2
for n in range(4, 10):
    path = Graph.subgraph_path_n(n)
    cycle = Graph.subgraph_cycle_n(n)
    complete = Graph.complete_n(n)

    assert(subgraph_search(cycle, path))
    assert(subgraph_search(complete, cycle))
    assert(subgraph_search(complete, path))

    # That left is not in right
    assert(not subgraph_search(path, cycle))
    assert(not subgraph_search(path, complete))
    assert(not subgraph_search(cycle, complete))